Применяя методы из предыдущих разделов, мы предполагали, что функция ценности дискретна и имеет конечное (относительно небольшое) число состояний. Пришло время избавиться от этого ограничения и обеспечить масштабируемый процесс обучения в средах с большим числом состояний и/или действий (возможно, даже несчетным). 

Естественно в такой ситуации вряд ли получится отыскать точное решение, поэтому придется аппроксимировать функции ценности.

Рабочим инструментом в этой серии будет среда MountainCar. У неё есть 2 непрерывных переменных состояния: положение в диапазоне от -1.2 (слева) до 0.6 (справа) и скорость, изменяющаяся от -0.07 до 0.07. Доступные действия: задний ход - 0, движение по инерции - 1 и передний ход - 2. Цель игры: добраться до вершины горы при условии, что двигатель имеет малую мощность. Агенту, чтобы достичь успеха, придётся отъезжать назад и разгоняться на спуске.

In [ ]:
!pip install gym pyvirtualdisplay atari-py piglet > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [ ]:
import gym

#env = gym.envs.make("MountainCar-v0")
env = wrap_env(gym.envs.make("MountainCar-v0"))

n_action = env.action_space.n
print(n_action)

env.reset()
env.render()
is_done = False
while not is_done:
    next_state, reward, is_done, info = env.step(2)
    #print(next_state, reward, is_done)
    env.render()

env.close()

3


In [ ]:
show_video()

Зная, что переменные состояния в данной среде непрерывны, попробуем линейно аппроксимировать Q-функцию с помощью градиентного спуска.

Сперва представим функцию ценности в виде взвешенного набор признаков: 

$V(s, \theta) = \theta_1 F_1(s) + \theta_2 F_2(s) + \ldots + \theta_n F_n(s) = \theta^T F(s)$

$F_i(s)$ -- набор признаков, $\theta_i$ -- соответствующие веса. 

Для линейного случая, который мы рассматриваем, схема действий особенно проста. Спева считаем градиент
$\nabla V(s, \theta) = F(s)$

Затем обновляем веса. Для TD-метода и среднеквадратичного отклонения функция обновления весов будет иметь вид
$\theta_{t+1} = \theta_t + \alpha [r + \gamma V(s_{t+1}) - V(s_t)] F(s) $

In [ ]:
import torch
from torch.autograd import Variable
import math


In [ ]:
class Estimator():
    def __init__(self, n_feat, n_state, n_action, lr=0.05):
        self.w, self.b = self.get_gaussian_wb(n_feat, n_state)
        self.n_feat = n_feat
        self.models = []
        self.optimizers = []
        self.criterion = torch.nn.MSELoss()

        for _ in range(n_action):
            model = torch.nn.Linear(n_feat, 1)
            self.models.append(model)
            optimizer = torch.optim.SGD(model.parameters(), lr)
            self.optimizers.append(optimizer)


    def get_gaussian_wb(self, n_feat, n_state, sigma=.2):
        """
        Generate the coefficients of the feature set from Gaussian distribution
        @param n_feat: number of features
        @param n_state: number of states
        @param sigma: kernel parameter
        @return: coefficients of the features
        """
        torch.manual_seed(0)
        w = torch.randn((n_state, n_feat)) * 1.0 / sigma        
        b = torch.rand(n_feat) * 2.0 * math.pi
        return w, b

    def get_feature(self, s):
        """
        Generate features based on the input state
        @param s: input state
        @return: features
        """
        features = (2.0 / self.n_feat) ** .5 * torch.cos(
            torch.matmul(torch.tensor(s).float(), self.w) + self.b)
        return features


    def update(self, s, a, y):
        """
        Update the weights for the linear estimator with the given training sample
        @param s: state
        @param a: action
        @param y: target value
        """
        features = Variable(self.get_feature(s))
        y_pred = self.models[a](features)

        loss = self.criterion(y_pred, Variable(torch.Tensor([y])))

        self.optimizers[a].zero_grad()
        loss.backward()
        self.optimizers[a].step()

    def predict(self, s):
        """
        Compute the Q values of the state using the learning model
        @param s: input state
        @return: Q values of the state
        """
        features = self.get_feature(s)
        with torch.no_grad():
            return torch.tensor([model(features) for model in self.models])

In [ ]:
  estimator = Estimator(10, 2, 1)
  s1 = [0.5, 0.1]
  print(estimator.get_feature(s1))

  s_list = [[1, 2], [2, 2], [3, 4], [2, 3], [2, 1]]
  target_list = [1, 1.5, 2, 2, 1.5]

  for s, target in zip(s_list, target_list):
      feature = estimator.get_feature(s)
      estimator.update(s, 0, target)
      
  print(estimator.predict([0.5, 0.1]))
  print(estimator.predict([2, 3]))